# SolarView

## Problema
Um clinte internacional da SolarView sabendo que temos um grande conhecimento em análise de energia em séries temporais pediu para realizarmos uma **análise exploratória de dados** para entender melhor sobre o proprio negocio. A ideia é **aplicarmos modelos de machine learning para realizar a previsão do consumo de energia em megawatts (MW)**. Porém ele enviou diversas bases de dados, precisamos com rapidez de uma análise, **o objetivo é enviar tanto um SQL quanto uma análise exploratória dos dados** para nosso pesquisador de machine learning dar segmento.

### Tarefa 1
Você deverá criar um banco de dados que vai conter todas as bases de dados, na pasta "data" você vai encontrar 14 arquivos .CSV. A forma que esses dados serão organizados vai depender de você, mas lembre de gestão colaborativa de dados. Outro funcionário da SolarView deverá conseguir criar tabelas atraves do seu SQL, ele não terá tempo de organizar os dados, todo processo deverá estar bem sucinto.

In [1]:
import pandas as pd
import os
import glob
import sqlite3
#import numpy as np

#### Leitura do arquivo do Apache Parquet 

In [2]:
from pyarrow.parquet import read_table
esthourlyTable = read_table('../data/est_hourly.paruqet')
type(esthourlyTable)

pyarrow.lib.Table

In [3]:
# Convertendo o arquivo para pandas
esthourly = esthourlyTable.to_pandas()
esthourly.head()

,AEP,COMED,DAYTON,DEOK,DOM,DUQ,EKPC,FE,NI,PJME,PJMW,PJM_Load
Datetime,,,,,,,,,,,,
1998-12-31 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29309.0
1998-12-31 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28236.0
1998-12-31 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27692.0
1998-12-31 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27596.0
1998-12-31 05:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27888.0


**Observação:** O arquivo "est_hourly.paruqet" consiste na primeira versão do Dataset, com os dados consolidados

#### Leitura do arquivo pjm_hourly_est.csv

In [4]:
pjm_hourly_est = pd.read_csv('../data/pjm_hourly_est.csv')
pjm_hourly_est.head()

,Datetime,AEP,COMED,DAYTON,DEOK,DOM,DUQ,EKPC,FE,NI,PJME,PJMW,PJM_Load
0,1998-12-31 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29309.0
1,1998-12-31 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28236.0
2,1998-12-31 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27692.0
3,1998-12-31 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27596.0
4,1998-12-31 05:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27888.0


**Observação:** O arquivo "pjm_hourly_est.csv" consiste na segunda versão do Dataset, com os dados consolidados

#### Leitura dos arquivos csv por região

In [5]:
# Lendo os arquivos CSV
path = r'E:\Dev\Source\DesafioSolarview\data'
allFiles = glob.glob(path + "/*.csv")

# Remove o arquivo pjm_hourly_est.csv
[allFiles.remove(f) for f in allFiles if 'pjm_hourly_est' in f]

# Lista com os Dataframes lidos
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_, parse_dates = ['Datetime'])
    list_.append(df)

for i in list_:
    print(i.head())

             Datetime   AEP_MW
0 2004-12-31 01:00:00  13478.0
1 2004-12-31 02:00:00  12865.0
2 2004-12-31 03:00:00  12577.0
3 2004-12-31 04:00:00  12517.0
4 2004-12-31 05:00:00  12670.0
             Datetime  COMED_MW
0 2011-12-31 01:00:00    9970.0
1 2011-12-31 02:00:00    9428.0
2 2011-12-31 03:00:00    9059.0
3 2011-12-31 04:00:00    8817.0
4 2011-12-31 05:00:00    8743.0
             Datetime  DAYTON_MW
0 2004-12-31 01:00:00     1596.0
1 2004-12-31 02:00:00     1517.0
2 2004-12-31 03:00:00     1486.0
3 2004-12-31 04:00:00     1469.0
4 2004-12-31 05:00:00     1472.0
             Datetime  DEOK_MW
0 2012-12-31 01:00:00   2945.0
1 2012-12-31 02:00:00   2868.0
2 2012-12-31 03:00:00   2812.0
3 2012-12-31 04:00:00   2812.0
4 2012-12-31 05:00:00   2860.0
             Datetime  DOM_MW
0 2005-12-31 01:00:00  9389.0
1 2005-12-31 02:00:00  9070.0
2 2005-12-31 03:00:00  9001.0
3 2005-12-31 04:00:00  9042.0
4 2005-12-31 05:00:00  9132.0
             Datetime  DUQ_MW
0 2005-12-31 01:00:00  1458.

In [6]:
# Verifica se existem valores NaN na coluna Datetime
for df in list_:
    print(df['Datetime'].isna().any())

False
False
False
False
False
False
False
False
False
False
False
False


In [7]:
# Merge dos Dataframes
def mergeFiles(left,right):
    df = pd.merge(left,right,on='Datetime',how='outer',sort=True)
    return df

df_final = pd.DataFrame(columns=['Datetime'])

for df in list_:
    df_final = mergeFiles(df_final, df)
    
df_final.head()

,Datetime,AEP_MW,COMED_MW,DAYTON_MW,DEOK_MW,DOM_MW,DUQ_MW,EKPC_MW,FE_MW,NI_MW,PJME_MW,PJMW_MW,PJM_Load_MW
0,1998-04-01 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22259.0
1,1998-04-01 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21244.0
2,1998-04-01 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20651.0
3,1998-04-01 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20421.0
4,1998-04-01 05:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20713.0


#### Preparação dos dados

Como os csv arquivos por região é a terceira (e mais recente) versão do Dataset, ele será considerado a versão final do Dataframe

In [8]:
# Verificando o tipo de dados das colunas
df_final.dtypes

Datetime       datetime64[ns]
AEP_MW                float64
COMED_MW              float64
DAYTON_MW             float64
DEOK_MW               float64
DOM_MW                float64
DUQ_MW                float64
EKPC_MW               float64
FE_MW                 float64
NI_MW                 float64
PJME_MW               float64
PJMW_MW               float64
PJM_Load_MW           float64
dtype: object

In [9]:
# Renomeando as colunas, removendo o final "_MW"
columns = df_final.columns.get_values().tolist()
columns = [name.replace("_MW", "") for name in columns]
df_final.columns = columns

In [10]:
# Verificando duplicidade na coluna Datetime
duplicate = pd.concat(g for _, g in df_final.groupby("Datetime") if len(g) > 1)
duplicate.groupby("Datetime").size()

Datetime
2014-11-02 02:00:00    1024
2015-11-01 02:00:00    1024
2016-11-06 02:00:00    1024
2017-11-05 02:00:00    1024
dtype: int64

In [11]:
# Remove registros duplicados utilizando a média
print(df_final.shape)
df_final = df_final.groupby('Datetime', as_index=False).mean()
print(df_final.shape)

(182350, 13)
(178258, 13)


In [12]:
# Alterando o index do Dataframe para a coluna Datetime
df_final = df_final.set_index('Datetime')

In [13]:
df_final.describe()

,AEP,COMED,DAYTON,DEOK,DOM,DUQ,EKPC,FE,NI,PJME,PJMW,PJM_Load
count,121269.000000,66493.000000,121271.000000,57735.000000,116185.000000,119064.000000,45330.000000,62870.000000,58450.000000,145362.000000,143202.000000,32896.000000
mean,15499.651721,11420.341585,2037.871556,3105.163029,10949.309171,1658.837277,1464.249217,7792.293017,11701.682943,32080.507722,5602.415755,29766.427408
std,2591.323421,2304.076173,393.392846,599.811175,2413.919516,301.731271,378.862987,1331.202720,2371.498701,6463.870519,979.125254,5849.769954
min,9581.000000,7237.000000,982.000000,907.000000,1253.000000,1014.000000,514.000000,0.000000,7003.000000,14544.000000,487.000000,17461.000000
25%,13630.000000,9781.000000,1749.000000,2687.000000,9323.000000,1444.000000,1185.000000,6807.000000,9954.000000,27573.000000,4907.000000,25473.000000
50%,15310.000000,11152.000000,2009.000000,3013.000000,10502.000000,1630.000000,1386.000000,7700.000000,11521.000000,31421.000000,5530.000000,29655.000000
75%,17200.000000,12510.000000,2279.000000,3449.000000,12378.000000,1819.000000,1699.000000,8556.000000,12896.750000,35650.000000,6252.000000,33073.250000
max,25695.000000,23753.000000,3746.000000,5445.000000,21651.000000,3054.000000,3490.000000,14032.000000,23631.000000,62009.000000,9594.000000,54030.000000


In [14]:
df_final.head()

,AEP,COMED,DAYTON,DEOK,DOM,DUQ,EKPC,FE,NI,PJME,PJMW,PJM_Load
Datetime,,,,,,,,,,,,
1998-04-01 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22259.0
1998-04-01 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21244.0
1998-04-01 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20651.0
1998-04-01 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20421.0
1998-04-01 05:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20713.0


In [15]:
df_final.tail()

,AEP,COMED,DAYTON,DEOK,DOM,DUQ,EKPC,FE,NI,PJME,PJMW,PJM_Load
Datetime,,,,,,,,,,,,
2018-08-02 20:00:00,17673.0,16437.0,2554.0,4052.0,14038.0,1966.0,1815.0,9866.0,NaN,44057.0,6545.0,NaN
2018-08-02 21:00:00,17303.0,15590.0,2481.0,3892.0,13832.0,1944.0,1769.0,9656.0,NaN,43256.0,6496.0,NaN
2018-08-02 22:00:00,17001.0,15086.0,2405.0,3851.0,13312.0,1901.0,1756.0,9532.0,NaN,41552.0,6325.0,NaN
2018-08-02 23:00:00,15964.0,14448.0,2250.0,3575.0,12390.0,1789.0,1619.0,8872.0,NaN,38500.0,5892.0,NaN
2018-08-03 00:00:00,14809.0,13335.0,2042.0,3281.0,11385.0,1656.0,1448.0,8198.0,NaN,35486.0,5489.0,NaN


#### Salvar o resultado do processamento dos dados

In [16]:
# Salvando como csv
df_final.to_csv('../output/df_final_v1.csv')

In [17]:
# Copia do Dataframe resetando o index
df_final2 = df_final.reset_index()
df_final2.head()

,Datetime,AEP,COMED,DAYTON,DEOK,DOM,DUQ,EKPC,FE,NI,PJME,PJMW,PJM_Load
0,1998-04-01 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22259.0
1,1998-04-01 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21244.0
2,1998-04-01 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20651.0
3,1998-04-01 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20421.0
4,1998-04-01 05:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20713.0


In [18]:
# Salvando no SQLite
pathSql = '../output/energy.db'
os.remove(pathSql) if os.path.exists(pathSql) else None
con = sqlite3.connect(pathSql)
#cur = con.cursor()
df_final2.to_sql(name='energy_consumption', con=con, index=False)
con.close()

In [19]:
# Salvando no MySql
import mysql.connector
import sqlalchemy

In [20]:
database_username = 'root'
database_password = 'toor'
database_ip       = '127.0.0.1'
database_name     = 'solarview'
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name), pool_recycle=1, pool_timeout=57600).connect()

df_final2.to_sql(con=database_connection, name='pjm_hourly', if_exists='append',chunksize=100)
database_connection.close()

#### Resumo

As colunas do dataset indica o **consumo de energia estimado em megawatts por hora** de diferentes companias de energia nos Estados Unidos.

Os dados consolidados compreendem o período de: **01/04/1998 01:00:00** a **03/08/2018 00:00:00**

* AEP - American Electric Power
* COMED - Commonwealth Edison
* DAYTON - The Dayton Power and Light Company
* DEOK - Duke Energy Ohio/Kentucky
* DOM - Dominion Virginia Power 
* DUQ - Duquesne Light Co.
* EKPC - East Kentucky Power Cooperative 
* FE - FirstEnergy
* NI - Northern Illinois Hub
* PJME - PJM East Region
* PJMW - PJM West Region
* PJM_Load - PJM Load Combined (Antes do split east e west)